In [ ]:
# Server
import socket
import json

# DHCP Server Configuration
server_ip_pool = ["192.168.1.100", "192.168.1.101", "192.168.1.102"]
leases = {}

# Start TCP Server to handle DHCP requests
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(("localhost", 65432))
server_socket.listen(5) # Allow up to 5 simultaneous connections
print("DHCP Server is running...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Client connected: {client_address}")

    # Receive and decode message from client
    data = client_socket.recv(1024).decode()
    message = json.loads(data)

    if message["type"] == "DISCOVER":
        print("\n[SERVER] Received DHCPDISCOVER")

        # Check if IPs are available
        if server_ip_pool:
            offered_ip = server_ip_pool.pop(0) # Assign the first available IP
            response = json.dumps({"type": "OFFER", "ip": offered_ip})
            client_socket.sendall(response.encode())
            print(f"[SERVER] Sent DHCPOFFER with IP {offered_ip}")
        else:
            client_socket.sendall(json.dumps({"type": "NACK"}).encode()) # No IPs available
            print("[SERVER] No IP available!")

    elif message["type"] == "REQUEST":
        print("\n[SERVER] Received DHCPREQUEST")
        requested_ip = message["ip"]
        leases[message["mac"]] = requested_ip  # Store the assigned IP for the MAC address
        response = json.dumps({"type": "ACK", "ip": requested_ip})
        client_socket.sendall(response.encode())
        print(f"[SERVER] Assigned {requested_ip} to {message['mac']}")
    
    client_socket.close()  # Close connection after response


DHCP Server is running...
Client connected: ('127.0.0.1', 55474)

[SERVER] Received DHCPDISCOVER
[SERVER] Sent DHCPOFFER with IP 192.168.1.100
Client connected: ('127.0.0.1', 55475)

[SERVER] Received DHCPREQUEST
[SERVER] Assigned 192.168.1.100 to AA:BB:CC:DD:EE:FF
